In [15]:
import collections
import json
import pathlib

In [16]:
jsons = pathlib.Path.cwd() / 'data' / 'jsons'
js = list(p for p in jsons.iterdir() if p.name.endswith('json'))
list(enumerate(js))

[(0,
  PosixPath('/Users/matteding/Desktop/Summarizer/data/jsons/2013-01-01__2013-12-31.json')),
 (1,
  PosixPath('/Users/matteding/Desktop/Summarizer/data/jsons/2014-01-01__2014-12-31.json')),
 (2,
  PosixPath('/Users/matteding/Desktop/Summarizer/data/jsons/2014-02-09__2014-12-31.json')),
 (3,
  PosixPath('/Users/matteding/Desktop/Summarizer/data/jsons/2013-06-16__2013-12-31.json'))]

In [16]:
file = js[0]

with open(file) as fp:
    dicts = []
    for line in fp.readlines():
        dct = json.loads(line)
        dicts.append(dct)
        
with open(file) as fp:
    lines = fp.readlines()

In [11]:
has_story = [d for d in dicts if d['story']]
no_story = [d for d in dicts if not d['story']] # one known was simply a map; see Jan/Feb 2010
len(has_story), len(no_story)

(1917, 0)

In [9]:
counter = collections.Counter(lines)
dupes = [json.loads(k) for k, v in counter.items() if v > 1]
dupes = [(j['date'], j['title']) for j in dupes]
len(counter), len(dupes)

(1841, 76)

### Merge 2010

In [17]:
# jsons = jsons / '2010'
# js = list(p for p in jsons.iterdir() if p.name.endswith('json'))
# list(enumerate(js))

[(0,
  PosixPath('/Users/matteding/Desktop/PyNotes/data/jsons/2010/2010-08-26__2010-12-31.json')),
 (1,
  PosixPath('/Users/matteding/Desktop/PyNotes/data/jsons/2010/2010-04-01__2010-12-31.json')),
 (2,
  PosixPath('/Users/matteding/Desktop/PyNotes/data/jsons/2010/2010-02-01__2010-03-31.json')),
 (3,
  PosixPath('/Users/matteding/Desktop/PyNotes/data/jsons/2010/2010-01-01__2010-01-01.json'))]

In [28]:
year = str(2010)
unique = set()

for path in (j for j in (jsons / year).iterdir() if j.name.endswith('json') and '_' in j.name):
    with open(path) as fp:
        unique.update(fp.readlines())
        
dicts = [json.loads(article) for article in unique]
merged = {year: [d for d in dicts if d['story']]}
filepath = jsons / year / f'{year}.json'
with open(filepath, 'w') as fp:
    json.dump(merged, fp)

In [29]:
len(unique), len(merged[year])

(4777, 4776)

In [30]:
[d for d in dicts if not d.get('story')]

[{'date': 'March 2, 2010 ',
  'title': 'Interactive Map: U.S. Rates Of Child Abuse Fatalities',
  'author': None,
  'summary': ' Using data from a report by the Department of Health and Human Services, Administration for Children & Families, this map chronicles child fatalities by maltreatment in most states. However, state data on child fatality from maltreatment is difficult to compare: There is no federal definition of child abuse, so cases are interpreted differently.',
  'story': ''}]

In [40]:
merged[year][4]

{'date': 'October 15, 2010 ',
 'title': 'Gates: Limited Damage From WikiLeaks Documents',
 'author': 'NPR Staff and Wires',
 'summary': ' In an Aug. 16 letter, the defense secretary said an initial Pentagon review showed the documents did not reveal any significant national intelligence secrets. But he conceded that the disclosure of the names of Afghan civilians with possible ties to the U.S. could pose a risk.',
 'story': 'The Pentagon says the documents leaked by the WikiLeaks website in July do not jeopardize any U.S. intelligence or sensitive military operations.\n\nIn an Aug. 16 letter released to NPR, Defense Secretary Robert Gates responded to questions about the WikiLeaks documents raised by Sen. Carl Levin (D-MI), chairman of the Senate Armed Services Committee.\n\nGates said an initial Pentagon review showed the documents focused mostly on day-to-day military operations and did not reveal any significant national intelligence secrets.\n\nThe letter confirmed that the documen

In [7]:
import shutil
import zipfile

In [14]:
year = str(2010)

file_to_zip = f'{year}.json'
target_path = jsons / year

compression = 'xztar'
try:
    shutil.make_archive(target_path / year, compression, target_path, file_to_zip)
except OSError:
    raise